<a href="https://colab.research.google.com/github/sj0110/AI_Beginners_Week05_Lab04/blob/main/MiniMax_pushkaraj_kolekar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
count = 0
alpha = -np.inf
beta = np.inf

class Game:
    
  # Initialize  the game by calling initialize_game method
    def __init__(self):
      
        self.initialize_game()
  # Initialize game board 
    def initialize_game(self):
        self.current_state = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]

        # Player X always plays first so set X as first player-human
        self.player_turn = 'X'
  # Method to print the current state of the board
    def draw_board(self):
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.current_state[i][j]), end=" ")
            print()
        print()
  # Method to check if the move played by player is valid
    def is_valid(self, px, py):
      # check for range of px and py
        if px < 0 or px > 2 or py < 0 or py > 2:
            return False
      # check if the current state is blank
        elif self.current_state[px][py] != '.':
            return False
        else:
            return True
  # Method to check if the game terminates
    def is_end(self):
            # Vertical win
        for i in range(0, 3):
            if (self.current_state[0][i] != '.' and
                self.current_state[0][i] == self.current_state[1][i] and
                self.current_state[1][i] == self.current_state[2][i]):
                return self.current_state[0][i]

            # Horizontal win
        for i in range(0, 3):
            if (self.current_state[i] == ['X', 'X', 'X']):
                return 'X'
            elif (self.current_state[i] == ['O', 'O', 'O']):
                return 'O'

            # Main diagonal win
        if (self.current_state[0][0] != '.' and
            self.current_state[0][0] == self.current_state[1][1] and
            self.current_state[0][0] == self.current_state[2][2]):
            return self.current_state[0][0]

            # Second diagonal win
        if (self.current_state[0][2] != '.' and
            self.current_state[0][2] == self.current_state[1][1] and
            self.current_state[0][2] == self.current_state[2][0]):
            return self.current_state[0][2]

        for i in range(0, 3): # Is whole board full?
            for j in range(0, 3):
                # There's an empty field, we continue the game
                if (self.current_state[i][j] == '.'):
                    return None
        # It's a tie!
        return '.'
    
    # Player 'O' is max, in this case AI
    def max(self):
        global count
        global alpha,beta
        # Possible values for maxv are:
        # Utility 
        # -1 : loss, 0  : a tie, 1  - win
        # We can initially set it to -2 as worse than the worst case for 3*3 matrix:
        # lower bound -generalised dependening upon m*m value
        maxv = -np.inf
        px = None
        py = None

        result = self.is_end()

        # If the game came to an end, the function needs to return
        # the evaluation function of the end. That can be:
        # -1 - loss, 0  - a tie, 1  - win
        # lose
        if result == 'X':
            return (-1, 0, 0)
        # win
        elif result == 'O':
            return (1, 0, 0)
        # draw
        elif result == '.':
            return (0, 0, 0)


        for i in range(0, 3):
            for j in range(0, 3):
                if self.current_state[i][j] == '.':
                    # On the empty field player 'O' makes a move and calls Min
                    # That's one branch of the game tree.
                    self.current_state[i][j] = 'O'
                    (m, min_i, min_j) = self.min()
                    count = count + 1
                    # Fixing the maxv value if needed
                    # maxv=-2
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    self.current_state[i][j] = '.'
                    alpha = max(alpha, m)
                    if beta <= alpha:
                        break
        return (maxv, px, py)

    # Player 'X' is min, in this case human
    def min(self):
        global count,alpha,beta
        # Possible values for minv are:
        # -1 - win, 0  - a tie, 1  - loss
        # We're initially setting it to 2 as worse than the worst case:
        
        minv = np.inf
        qx = None
        qy = None
        # check if game terminates
        result = self.is_end()
        # win
        if result == 'X':
            return (-1, 0, 0)
        # lose
        elif result == 'O':
            return (1, 0, 0)
        # tie
        elif result == '.':
            return (0, 0, 0)
        
        for i in range(0, 3):
            for j in range(0, 3):
                if self.current_state[i][j] == '.':
                    self.current_state[i][j] = 'X'
                    # calling max method
                    (m, max_i, max_j) = self.max()
                    count = count + 1
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.current_state[i][j] = '.'
                    beta = min(beta, m)
                    if beta <= alpha:
                      break
        return (minv, qx, qy)

    def play(self):
        global count,alpha,beta
        while True:
            self.draw_board()
            self.result = self.is_end()
            if self.result != None:
                if self.result == 'X':
                    print('The winner is X!')
                elif self.result == 'O':
                    print('The winner is O!')
                elif self.result == '.':
                    print("It's a tie!")

                self.initialize_game()
                return

            # If it's player's turn
            if self.player_turn == 'X':

                while True:
                    # for evaluation time
                    start = time.time()
                    (m, qx, qy) = self.min()
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Recommended move: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('Insert the X coordinate: '))
                    py = int(input('Insert the Y coordinate: '))

                    (qx, qy) = (px, py)
                    
                    if self.is_valid(px, py):
                        self.current_state[px][py] = 'X'
                        self.player_turn = 'O'
                        break
                    else:
                        print('The move is not valid! Try again.')

            # If it's AI's turn
            else:
                (m, px, py) = self.max()
                self.current_state[px][py] = 'O'
                self.player_turn = 'X'
             
           

def main():
    global count,alpha,beta
    g = Game()
    g.play()
    print("\n Number of node visited for this states: ", count)
   

if __name__ == "__main__":
    global count,alpha,beta
    main()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.016901s
Recommended move: X = 0, Y = 0
